In [1]:
# load the sheet
import pandas as pd
import numpy as np

class MageSheetProductLoader:
    def __init__(self):
        pass

    def _sanitize_df(self, product_df):
        """Chuck all dep"""
        # product_df = product_df[~product_df.index.str.contains("_DEP")]
        product_df = product_df[~product_df.index.str.contains("webpos_")]
        return product_df
    
    def fetch_products(self):
        sheet_db = pd.read_csv("/Users/varunpasad/projects/apps/assets/asserts/export_product.csv")
        sheet_db = sheet_db.set_index("sku")
        
        return self._sanitize_df(sheet_db)


In [2]:
# build the base sheet
sheet_db = MageSheetProductLoader().fetch_products()
# cost db
cost_db = pd.read_csv("/Users/varunpasad/projects/apps/assets/asserts/pos_price.csv")
cost_db = cost_db.set_index("sku")[['price']]
# print(len(sheet_db))
sheet_db = sheet_db.join(cost_db, how='left')
# print (sheet_db)

odoo_products = sheet_db.reset_index()[['sku', 'name', 'price']]
odoo_products = odoo_products.rename(columns={
    'sku': 'default_code', 'price':'standard_price'})
odoo_products = odoo_products.set_index('default_code')

In [3]:
# GEnerate catalog
SKU_TO_CAT = {
    "CHK_": "tcuts.product_category_chicken",
    "GT_": "tcuts.product_category_goat",
    "LAMB_": "tcuts.product_category_goat",
    "SF_": "tcuts.product_category_seafood",
    "MRT_":  "tcuts.product_category_marinades",
    "YUMMIEZ":  "tcuts.product_category_yummiez",
    
    "RM_CHK_": "tcuts.product_category_chicken",
    "RM_GT_": "tcuts.product_category_goat",
    "RM_MUT_": "tcuts.product_category_goat",
    "RM_LAMB_": "tcuts.product_category_goat",
    "RM_SF_" : "tcuts.product_category_seafood",
    "RM_MRT_": "tcuts.product_category_marinades",
    "RM_YUMMIEZ":  "tcuts.product_category_yummiez",
}

def is_raw_material(sku):
    if sku.startswith("RM_"):
        return True
    
    return False


def build_row(row):
    sku = row['default_code']
    is_raw = is_raw_material(sku)

    for part, cat_name in SKU_TO_CAT.items():
        if sku.startswith(part):
            row['categ_id/id'] = cat_name
            break

    # SKU
    if not is_raw:
        row['purchase_ok'] = "FALSE"
        row['sale_ok'] = "TRUE"
        row['use_time'] = 2
        row['alert_time'] = 1
        row['removal_time'] = 2
        row['life_time'] = 2
        row['uom_po_id/id'] = "product.product_uom_kgm"
        row['uom_id/id'] = "product.product_uom_kgm"
        row['tracking']  = "none"
    else:
        row['purchase_ok'] = "TRUE"
        row['sale_ok'] = "TRUE"
        row['use_time'] = 3
        row['alert_time'] = 2
        row['removal_time'] = 3
        row['life_time'] = 4
        row['uom_po_id/id'] = "product.product_uom_kgm"
        row['uom_id/id'] = "product.product_uom_kgm"
        row['tracking']  = "lot"
        
    # no taxes
    row['taxes_id'] = None
    row['supplier_taxes_id'] = None

    return row

def generate_sheet(sheet_db):
    # external id
    sheet_db['type']  = "Stockable Product"
    sheet_db = sheet_db.reset_index().apply(build_row, axis=1)

    # convert _DEP to inactive in odoo and build the ID
    sheet_db['active'] = np.where(sheet_db.default_code.str.endswith("_DEP"), "FALSE", "TRUE")
    sheet_db.default_code = sheet_db.default_code.str.replace(r"^(.*)_DEP$", r"\1")

    sheet_db = sheet_db.set_index("default_code")
    sheet_db['id'] = "tcuts.product_" + sheet_db.index
    sheet_db = sheet_db[~sheet_db['categ_id/id'].isnull()]
    
    # set active/inactive
    
    
    print (sheet_db[sheet_db['categ_id/id'].isnull()])
    sheet_db.reset_index().to_csv(
        "/Users/varunpasad/projects/odoo/tcuts/data/product.template.csv",
        index=False,
        columns=["id", "default_code", "name", "type", "tracking", "categ_id/id",
                 "uom_po_id/id", "uom_id/id", "purchase_ok", "sale_ok",
                "use_time", "alert_time", "removal_time", "life_time",
                "taxes_id", "supplier_taxes_id", "standard_price", "active"]
    )
    print (len(sheet_db))
    return sheet_db

df = generate_sheet(odoo_products)


Empty DataFrame
Columns: [alert_time, categ_id/id, life_time, name, purchase_ok, removal_time, sale_ok, standard_price, supplier_taxes_id, taxes_id, tracking, type, uom_id/id, uom_po_id/id, use_time, active, id]
Index: []
323


In [4]:
# generate BOm
sheet_db = MageSheetProductLoader().fetch_products()
sheet_db = sheet_db[~sheet_db.index.str.startswith("RM_")]


# Prepare the bom sheet
bom_db = sheet_db.reset_index()[['sku', 'parent_sku', 'gramsperunit']]

# convert _DEP to inactive in odoo and build the ID
bom_db['active'] = np.where(bom_db.sku.str.endswith("_DEP"), "FALSE", "TRUE")
bom_db.sku = bom_db.sku.str.replace(r"^(.*)_DEP$", r"\1")
bom_db.parent_sku = bom_db.parent_sku.str.replace(r"^(.*)_DEP$", r"\1")

bom_db = bom_db.rename(columns={
    "sku": 'product_tmpl_id/id',
    "parent_sku": "bom_line_ids/product_id/id",
    "gramsperunit": "bom_line_ids/product_qty"
})

# Hack to introduce 1:1 ratio
bom_db["bom_line_ids/product_qty"] = 1000

# generate Ids
bom_db['id'] = "tcuts.mrp_bom_" + bom_db['product_tmpl_id/id']

# GEnerate ID for the BOM lines
# only one BOM is ther currently so the hack of _1
bom_db['bom_line_ids/id'] = "tcuts.bom_line_" + bom_db['product_tmpl_id/id'] + "_1"

# Format the SKU to External ID
bom_db['bom_line_ids/product_id/id'] = "tcuts.product_" + bom_db['bom_line_ids/product_id/id']
bom_db['product_tmpl_id/id'] = "tcuts.product_" + bom_db['product_tmpl_id/id']

bom_db['product_qty'] = 1
bom_db['product_uom_id/id'] = "product.product_uom_kgm"
bom_db['type'] = "Manufacture this product"

# BOM LINES dataa:
# convert to kg
bom_db['bom_line_ids/product_qty'] /= 1000 
bom_db['bom_line_ids/product_uom_id/id'] = "product.product_uom_kgm"

# filter out crappy records
bom_db = bom_db[~bom_db["bom_line_ids/product_qty"].isnull()]
bom_db = bom_db[~bom_db["bom_line_ids/product_id/id"].isnull()]

# split the df into 2 sheets mrp.bom.line and mrp.bom
bom_db.reset_index().to_csv(
        "/Users/varunpasad/projects/odoo/tcuts/data/mrp.bom.csv",
        index=False,
        columns=["id", "product_tmpl_id/id", "product_qty", "type", "active", "product_uom_id/id"]
    )

bom_db.to_csv(
        "/Users/varunpasad/projects/odoo/tcuts/data/mrp.bom.line.csv",
        index=False,
        header=["id", "product_id/id", "product_qty", "product_uom_id/id", "bom_id/id"],
        columns=[ "bom_line_ids/id",
                 "bom_line_ids/product_id/id", "bom_line_ids/product_qty",
                 "bom_line_ids/product_uom_id/id",
                 "id"]
    )
    


In [ ]:
/